In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from scipy import stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

In [2]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not    
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [29]:
subject_dfs = {}
for x in tqdm(rlist):
    subject_dfs[x] = pd.read_csv(os.path.normpath('mit-bih-dataframes/'+x+'.csv'), index_col=0)

100%|██████████| 23/23 [01:30<00:00,  3.91s/it]


In [27]:
feature_dfs = {}
for x in tqdm(rlist):
    feature_dfs[x] = pd.read_csv(os.path.normpath('mit-bih-time-features/'+x+'.csv'), index_col=0)

100%|██████████| 23/23 [00:01<00:00, 21.41it/s]


In [30]:
count_dict = {
    'subjectID': [],
    'total_ints': [],
    'normal_freq': [],
    'afib_freq': [],
    'other_freq': []
}
for subject in tqdm(rlist):
    subject_df = subject_dfs[subject]
    count_dict['subjectID'].append(subject)
    count_dict['total_ints'].append(len(subject_df))

    length = len(subject_df)
    normal_count = len(subject_df[subject_df['Normal']==True])
    afib_count = len(subject_df[subject_df['AFIB']==True])
    other_count = len(subject_df[subject_df['Other']==True])

    count_dict['normal_freq'].append(np.round(normal_count/length*100, 3))
    count_dict['afib_freq'].append(np.round(afib_count/length*100, 3))
    count_dict['other_freq'].append(np.round(other_count/length*100, 3))

count_df = pd.DataFrame(count_dict).set_index('subjectID')
count_df

100%|██████████| 23/23 [00:13<00:00,  1.65it/s]


,total_ints,normal_freq,afib_freq,other_freq
subjectID,,,,
04015,9205760,99.355,0.644,0.000
04043,9205760,78.457,21.507,0.036
04048,9205760,99.018,0.980,0.002
04126,9205760,96.256,3.742,0.002
04746,9205760,46.899,53.100,0.001
04908,9205760,90.940,8.371,0.689
04936,9205760,18.660,72.131,9.209
05091,9205760,99.764,0.236,0.001
05121,9205760,36.347,63.028,0.624
